In [ ]:
import napari
import pandas as pd
from napari_skimage_regionprops import regionprops_table, add_table, get_table
import numpy as np

In [ ]:
'''
Function to get IDs of cells present in a certain % planes 
can be used for example to pinpoint the best / fully segmented ones 
or the truncated ones that need correcting! 
'''

def select_cells_by_integrity(z_planes, cells, percentage, analysis_type):
    target_n_planes = round ((percentage / 100 ) * z_planes)
    selected_cells = []
    if analysis_type not in ["good","bad"]:
        raise ValueError('Invalid input. Please choose either "good" or "bad" cells.')
    for i in range(0,len(cells)):
        if analysis_type == 'good' and cells[i][1] >= target_n_planes:
                selected_cells.append(cells[i])
        elif analysis_type == 'bad' and cells[i][1] <= target_n_planes:
                selected_cells.append(cells[i])
    return selected_cells

# function to filter labels based on a list of label ids one wants to keep (one frame)
def select_labels_oneframe(input_labels, ids_to_keep):
    mask = np.isin(input_labels, ids_to_keep)
    selected_labels = np.where(mask, input_labels, 0)
    return selected_labels

# function to filter labels based on a list of label ids one wants to keep (for a stack/movie)
def select_labels_stack(input_labels, ids_to_keep):
    picked_labels = []
    for i in range(0,len(input_labels.data)):
        current_frame = (input_labels.data[i])
        picked_labels.append(select_labels_oneframe(current_frame, ids_to_keep)) 
    return picked_labels


In [ ]:
viewer = napari.Viewer()

***
In the Napari viewer, open the image + segmented data to evaluate, convert to 2D timelapse and then use the "regionprops of all frames" plugin to get the output table\
Usually there will be two layers open: the labels layer as top layer and the image/data layer below
***

In [ ]:
# Get the regionprops table, associated to the labels layer
labels = viewer.layers[1]
table = get_table(labels, viewer)

# read the content from the table as a dictionary. It is recommended to convert it into a pandas DataFrame:
regionprops_output = pd.DataFrame(table.get_content())

# count the number of frames in which each label (cell) appears and create array with [label, count]
labels = regionprops_output['label']
labels_nframes=[]
for i in range(0,regionprops_output['label'].nunique()):
    labels_nframes.append([regionprops_output['label'][i],len(regionprops_output[regionprops_output['label']==regionprops_output['label'][i]].frame)])

In [ ]:
# extract both "good" (present in at least 85% planes) and "bad" (present in fewer than 15% planes) cell labels  
labels_layer = viewer.layers[1]
image_layer = viewer.layers[0]

good_cells = select_cells_by_integrity(len(image_layer.data), labels_nframes, 85, 'good')
good_ids = [x[0] for x in good_cells]
good_labels = select_labels_stack(labels_layer, good_ids)

bad_cells = select_cells_by_integrity(len(image_layer.data), labels_nframes, 15, 'bad')
bad_ids = [x[0] for x in bad_cells]
bad_labels = select_labels_stack(labels_layer, bad_ids)

print('Percentage of "good" labels in dataset =',round((len(good_cells)/len(labels_nframes))*100,1),'%')
print('Percentage of "bad" labels in dataset =',round((len(bad_cells)/len(labels_nframes))*100,1),'%')

In [ ]:
# add the new labels to the viewer
viewer.add_labels((np.array(good_labels)), name='good cells')
viewer.add_labels((np.array(bad_labels)), name='bad cells')

In [ ]:
# Note: if labels need to be 3D instead of timelapse they can be reshaped
label = good_labels
label_3d = np.reshape(label, (label.shape[1], label.shape[0], label.shape[2], label.shape[3])) 